# **Named Entity Recognition**

## **1- Introduction**

In this section, we demonstrate how to implement named entity recognition using spaCy.

SpaCy is one of the most famous framework for NLP. It can be used for the implementation of tasks for sentiment analysis, chatbots, text summarization, intent and entity extraction, and others.

More information about spaCy please refer to [[1]](#scrollTo=op-j6UywUt5i).

### **Content**
In this notebook, some basic examples of named entity recognition will be shown.


## **2- Named Entity Recognition (NER)**

Named entity recognition (NER) is the task of locating and classifying
named entities mentioned in unstructured text into predefined categories such as
names, organizations, locations, medical codes, time expressions, quantities, monetary values, and percentages. 

NER provides important information to understand the content of a text, and is an excellent starting point for all kinds of text analysis and data organization. NER is a token classification problem.

### **Code Examples**

For named entity recognition , we will follow the following steps:
* Import the spaCy library
* Load the language model (English)
* Create a spaCy document
* Access the POS tags by iterating over the document object
* Print the POS tags

In [ ]:
# Load resources for all following code cells
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
sp = spacy.load('en_core_web_sm')

In [ ]:
# Create NER document
doc_ner = sp(u'Christiano Ronaldo was signed by Juventus for $105 million')

for entity in doc_ner.ents:
    print(entity.text + ' - ' 
          + entity.label_ + ' - ' 
          + str(spacy.explain(entity.label_)))

Christiano Ronaldo - PERSON - People, including fictional
Juventus - ORG - Companies, agencies, institutions, etc.
$105 million - MONEY - Monetary values, including unit


## **3- References**

- [1] https://spacy.io/

Copyright © 2021 IU International University of Applied Sciences